In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
import robot as Robot
import  time

robot_Instance= Robot.Robot_Class(0,0,0)


In [ ]:
robot_Instance.Connect_TCP()

Trying to connect


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 3291, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-2ec41826f6a7>", line 1, in <module>
    robot_Instance.Connect_TCP()
  File "/home/pi/STI_Robotic_Competition_Software/Architecture/robot.py", line 55, in Connect_TCP
    self.TCP_Conn, self.TCP_Addr, self.TCP_S = Tcp_Ip.enable_tcp_com(BUFFER_SIZE)
  File "/home/pi/STI_Robotic_Competition_Software/Architecture/TCP_IP.py", line 12, in enable_tcp_com
    conn, addr = s.accept()
  File "/usr/lib/python3.5/socket.py", line 195, in accept
    fd, addr = self._accept()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py", line 2033, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'KeyboardInterrupt' object 

In [4]:
x,y,a=robot_Instance.Get_Beacon_Position()
robot_Instance.Set_Robot_Position(x,y,a,False)


Time to get image:  1.9662554264068604
First batch circle:  0.26519107818603516
Second batch circle:  0.853722095489502
([123.50343698241423, -10.420712396794464, -157.7824057304817, -158.00259659335785], [(0, 8000), (8000, 0), (0, 0), (8000, 8000)])
Find angle:  1.9491112232208252
Find robot pos:  0.8131959438323975
Beacon worked!
total time post amth:  6.012092113494873
Beacon (3731.192327951506, 1166.4769090630514, -4.868323675457566)


In [36]:
Init=[1000,1000,90,15]
Spot_Init=[Init[0],Init[1],0,15]
robot_Instance.Set_Robot_Position(Init[0],Init[1],Init[2],Transmission=True)
robot_Instance.Set_Robot_Speed(500, Transmission= True)
robot_Instance.Set_Max_Try_Single_Goal(12)

G92 X1000 Y1000 A90 



In [37]:
robot_Instance.Empty_TCP()
Spot_List =[Spot_Init,[1000,2600,0,15],[6500,2600,0,15],[6500,5000,0,15],[4500,5000,0,15],[4500,7500,0,15],[4500,5000,0,15],[7500,5000,0,15],[7500,500,0,15],[2500,500,0,15],[2500,4000,0,15],[7000,4000,0,15],[7000,1500,0,15],[3500,1500,0,15],[700,3500,0,15],[5500,3500,0,15],[5500,4200,0,15],[700,4200,0,15]]
for Spot in Spot_List:
    Bottle_N=0
    robot_Instance.Sensor_Enabled=Spot[3]
    print('Spot : ', Spot)
    robot_Instance.Clear_Read_Buffer()
    robot_Instance.Clear_Bottle_List()
    robot_Instance.Go_To(Spot[0],Spot[1],Spot[2])
    #Robot is now arrived to a position
    Data=robot_Instance.Get_Data_TCP()#Collect bottles from TCP Communication
    print('Data Received TCP:', Data)
    while Bottle_N<3:#Try to catch 3 Bottle
        List=robot_Instance.Update_Bottle_List(Data)#Update our internal lsit
        Bottle_List= robot_Instance.Get_Bottle_List()
        if Bottle_List!=[]:
            Bottle_X, Bottle_Y, Bottle_Score= robot_Instance.Get_Nearest_Bottle()
            if Bottle_X!=None and Bottle_Y!= None and Bottle_Score!=None:
                Bottle_X_Extended, Bottle_Y_Extended, Valid=robot_Instance.Extend_Coordinate((Bottle_X, Bottle_Y),Distance=800)
                print('Bottle selected:', Bottle_X,Bottle_Y,Bottle_Score, ' Extended', Bottle_X_Extended,Bottle_Y_Extended)
                if Valid:
                    robot_Instance.Go_To(Bottle_X,Bottle_Y,Bottle_Score)
                    robot_Instance.Add_Bottle_Collected()
                    Bottle_N=Bottle_N+1
                else:
                    robot_Instance.Clean_Bottle_Of_Position((Bottle_X, Bottle_Y))
            else:# No more bottle
                print('No bottle')
                break;
        else:
            print("Empty List")
            break;
    if(robot_Instance.Get_Bottle_Collected()>8):
        robot_Instance.Go_To(2000,2000,1)
        robot_Instance.Go_To(1000,1000,0)
        robot_Instance.Go_To(100,100,0, Avoidance=False)
        #Reverse collector
        robot_Instance.Reset_Bottle_Collected()


Unconnected
Spot :  [1000, 1000, 0, 15]
Unconnected
Data Received TCP: None
Empty List
Spot :  [1000, 2600, 0, 15]
{'h': 0, 'k': 0, 'e': 0, 'i': 0, 'f': 0, 'j': 0, 'b': 0, 'm': 0, 'c': 0, 'g': 0, 'l': 0, 'd': 0, 'a': 0}
Message  M0 X989.63 Y2457.89 A86.93 T1 S0

Timeout
Unconnected
Data Received TCP: None
Empty List
Spot :  [6500, 2600, 0, 15]
{'h': 0, 'k': 0, 'e': 0, 'i': 0, 'f': 0, 'j': 0, 'b': 0, 'm': 0, 'c': 0, 'g': 0, 'l': 1, 'd': 0, 'a': 0}
Message  M0 X4644.67 Y2819.96 A-4.66 T2 S2048

Blocked
{'h': 0, 'k': 0, 'e': 0, 'i': 0, 'f': 0, 'j': 0, 'b': 0, 'm': 0, 'c': 0, 'g': 0, 'l': 0, 'd': 0, 'a': 0}
Message  M0 X4515.98 Y2839.94 A-4.81 T1 S0

Timeout
{'h': 0, 'k': 0, 'e': 0, 'i': 0, 'f': 0, 'j': 0, 'b': 0, 'm': 0, 'c': 0, 'g': 0, 'l': 0, 'd': 0, 'a': 0}
Message  M0 X4637.24 Y2929.74 A64.68 T1 S0

Timeout
{'h': 0, 'k': 0, 'e': 0, 'i': 0, 'f': 0, 'j': 0, 'b': 0, 'm': 0, 'c': 0, 'g': 0, 'l': 0, 'd': 0, 'a': 0}
Message  M0 X6330.12 Y2681.36 A-23.27 T1 S0

Timeout
Unconnected
Data Recei

{'h': 0, 'k': 0, 'e': 0, 'i': 0, 'f': 0, 'j': 0, 'b': 0, 'm': 0, 'c': 0, 'g': 0, 'l': 1, 'd': 0, 'a': 0}
Message  M0 X7255.23 Y1153.45 A290.39 T2 S2048

Blocked
{'h': 0, 'k': 0, 'e': 0, 'i': 0, 'f': 0, 'j': 0, 'b': 0, 'm': 0, 'c': 0, 'g': 0, 'l': 0, 'd': 0, 'a': 0}
Message  M0 X7210.38 Y1293.35 A288.75 T1 S0

Timeout
{'h': 0, 'k': 0, 'e': 0, 'i': 0, 'f': 0, 'j': 0, 'b': 0, 'm': 0, 'c': 0, 'g': 0, 'l': 0, 'd': 0, 'a': 0}
Message  M0 X7321.49 Y1199.28 A346.62 T1 S0

Timeout
{'h': 0, 'k': 0, 'e': 0, 'i': 0, 'f': 0, 'j': 0, 'b': 0, 'm': 0, 'c': 0, 'g': 0, 'l': 0, 'd': 0, 'a': 0}
Message  M0 X7447.33 Y1208.93 A380.15 T1 S0

Timeout
{'h': 0, 'k': 0, 'e': 0, 'i': 0, 'f': 0, 'j': 0, 'b': 0, 'm': 0, 'c': 0, 'g': 0, 'l': 0, 'd': 0, 'a': 0}
Message  M0 X7422.29 Y1201.02 A356.97 T1 S0

Timeout
{'h': 0, 'k': 0, 'e': 0, 'i': 0, 'f': 0, 'j': 0, 'b': 0, 'm': 0, 'c': 0, 'g': 0, 'l': 0, 'd': 0, 'a': 0}
Message  M0 X7423.52 Y1200.95 A356.55 T1 S0

Timeout
{'h': 0, 'k': 0, 'e': 0, 'i': 0, 'f': 0, 'j': 0, 

KeyboardInterrupt: 

In [18]:
Init=[1000,1000,0,15]
Spot_Init=[Init[0],Init[1],Init[2], Init[3]]
robot_Instance.Set_Robot_Position(Init[0],Init[1],Init[2],Transmission=True)
time.sleep(0.1)
robot_Instance.Set_Robot_Speed(500, Transmission= True)
time.sleep(0.1)

robot_Instance.Set_Max_Try_Single_Goal(12)
robot_Instance.Empty_TCP()
Spot_List =[Spot_Init,[4000,1000,0,15]]
for Spot in Spot_List:
    Bottle_N=0
    robot_Instance.Sensor_Enabled=Spot[3]
    robot_Instance.Go_To(Spot[0],Spot[1],Spot[2], Avoidance=True)
    print(Spot)


Unconnected
[1000, 1000, 0, 15]
OK: X=7500.00 Y=7700.00 A=90.00 

{'b': 0, 'f': 0, 'c': 0, 'h': 0, 'g': 0, 'm': 0, 'd': 0, 'e': 0, 'l': 0, 'a': 0, 'k': 0, 'j': 0, 'i': 0}
 M0 X7163.50 Y7586.40 A190.61 T1 S0

Timeout
OK: X=1000.00 Y=1000.00 A=0.00 

OK: D_Profil S=500 A=50 B=100 

{'b': 0, 'f': 0, 'c': 0, 'h': 0, 'g': 0, 'm': 0, 'd': 0, 'e': 0, 'l': 1, 'a': 0, 'k': 0, 'j': 0, 'i': 0}
 M0 X2959.17 Y1032.59 A1.07 T2 S2048

Blocked
{'b': 0, 'f': 0, 'c': 0, 'h': 0, 'g': 0, 'm': 0, 'd': 0, 'e': 0, 'l': 0, 'a': 0, 'k': 0, 'j': 0, 'i': 0}
 M0 X2820.51 Y1037.50 A1.10 T1 S0

Timeout
{'b': 0, 'f': 0, 'c': 0, 'h': 0, 'g': 0, 'm': 0, 'd': 0, 'e': 0, 'l': 0, 'a': 0, 'k': 0, 'j': 0, 'i': 0}
 M0 X2940.97 Y1142.29 A73.55 T1 S0

Timeout
{'b': 0, 'f': 0, 'c': 0, 'h': 0, 'g': 0, 'm': 0, 'd': 0, 'e': 0, 'l': 0, 'a': 0, 'k': 0, 'j': 0, 'i': 0}
 M0 X3780.48 Y1124.67 A-27.18 T1 S0

Timeout
{'b': 0, 'f': 0, 'c': 0, 'h': 0, 'g': 0, 'm': 0, 'd': 0, 'e': 0, 'l': 0, 'a': 0, 'k': 0, 'j': 0, 'i': 0}
 M0 X3914.76 Y10

In [20]:
robot_Instance.Go_To_Speed(150,150,3000, Detection=False)

('G1 R150 L150 T3000\r\n', 'M3 H2 S0\r\n')
OK: X=0.00 Y=0.00 A=0.00 

 M0 H2 T2

Blocked


In [3]:
Init=[7500,3000,90,15]
robot_Instance.Set_Robot_Position(Init[0],Init[1],Init[2],Transmission=True)
robot_Instance.Start_Collector()
robot_Instance.Set_Robot_Speed(500, Transmission= True)
robot_Instance.Clear_Read_Buffer()
"""robot_Instance.Go_To_Speed(600,640,20000, Detection=True)#Follow X coordinate
robot_Instance.Go_To_Speed(600,640,20000, Detection=True)
robot_Instance.Go_To_Speed(100,100,800,X_Cal=7600,Y_Cal=200,A_Cal=0, Detection=False, Calibration=True)#Recalibrate Top of the ramp
robot_Instance.Go_To(7000,200,1,Path_Checking=False,Avoidance=False)#Backward
robot_Instance.Go_To(7800,1000,0,Path_Checking=False,Avoidance=False)#Turn left"""

"""robot_Instance.Go_To_Speed(600,640,20000, Detection=True)#Follow Y coordintae
"""
robot_Instance.Go_To_Speed(400,440,20000, Detection=True)
print(robot_Instance.Get_Robot_Position())

robot_Instance.Go_To_Speed(100,100,800,X_Cal=7500,Y_Cal=7700,A_Cal=90, Detection=False, Calibration=True)#Recalibrate Top of the ramp, Step1
robot_Instance.Disable_All_Sensors()
robot_Instance.Go_To(7500,7200,1,Path_Checking=False,Avoidance=False)#Backward
robot_Instance.Go_To(7000,7200,0,Path_Checking=False,Avoidance=False)#Move backward right so we can recalibrate at the top of the ramp
robot_Instance.Go_To_Speed(-320,-300,10000, Detection=True)#Go against wall
robot_Instance.Go_To_Speed(-100,-100,1500,X_Cal=7900,A_Cal=180, Detection=False, Calibration=True)#Recalibrate Corner paltform
robot_Instance.Go_To(5400,7500,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(5400,7000,0,Path_Checking=False,Avoidance=False)#In front of stairs
robot_Instance.Go_To(7000,6500,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7000,7600,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(5400,7000,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(5400,7600,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7800,7600,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To_Speed(420,400,20000, Detection=True)#Calibrate Top of the ramp
robot_Instance.Go_To_Speed(100,100,1200,X_Cal=7700,Y_Cal=7750,A_Cal=0, Detection=False, Calibration=True)

robot_Instance.Go_To(7300,7750,1,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7600,7600,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7800,7300,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To_Speed(120,100,10000, Detection=True)#Downhill
robot_Instance.Go_To_Speed(640,600,20000, Detection=True)
robot_Instance.Go_To_Speed(100,100,800,X_Cal=7800,Y_Cal=300,A_Cal=-90, Detection=False, Calibration=True)
robot_Instance.Go_To(7800,700,1,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7000,700,1,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To_Speed(640,600,20000, Detection=True)#Recalibrate Origin
robot_Instance.Go_To_Speed(640,600,20000, Detection=True)#Recalibrate Origin

robot_Instance.Go_To_Speed(100,100,800,X_Cal=200,Y_Cal=300,A_Cal=180, Detection=False, Calibration=True)
robot_Instance.Set_Robot_Speed(150, Transmission= True)

robot_Instance.Reverse_Collector()
robot_Instance.Clear_Read_Buffer()
robot_Instance.Go_To(1400,300,1,Path_Checking=False,Avoidance=False)
time.sleep(5)
robot_Instance.Stop_Collector()
robot_Instance.Go_To(1700,300,1,Path_Checking=False,Avoidance=False)
"""End of RAMP"""
robot_Instance.Start_Collector()

robot_Instance.Set_Robot_Speed(500, Transmission= True)
time.sleep(0.2)
robot_Instance.Clear_Read_Buffer()

robot_Instance.Go_To(1700,750,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7000,750,0,Path_Checking=False,Avoidance=True)
robot_Instance.Go_To(7000,1500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Go_To(500,1500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Go_To(500,1500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Go_To(500,1500,0,Path_Checking=False,Avoidance=True)
robot_Instance.Go_To_Speed(200,200,20000, Detection=True)#Recalibrate Origin
robot_Instance.Go_To_Speed(100,100,800,X_Cal=400,Y_Cal=3000,A_Cal=-180, Detection=False, Calibration=True)
robot_Instance.Go_To(1000,3000,1,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(1000,2500,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To_Speed(400,440,20000, Detection=True)#Recalibrate Origin
robot_Instance.Go_To_Speed(100,100,800,Y_Cal=400,A_Cal=-90, Detection=False, Calibration=True)
robot_Instance.Set_Robot_Speed(150, Transmission= True)
robot_Instance.Reverse_Collector()
robot_Instance.Clear_Read_Buffer()
robot_Instance.Go_To(1000,1000,1,Path_Checking=False,Avoidance=False)
time.sleep(5)
robot_Instance.Stop_Collector()
"""End of Grass"""


G92 X7500 Y3000 A90 

('G1 R400 L440 T20000\r\n', 'M3 H2 S15\r\n')
PreParse OK: D_Profil S=500 A=50 B=100 

Message OK: D_Profil S=500 A=50 B=100 

PreParse  M0 H2 T2

PostParse <Result ('2',) {}>
Message  M0 H2 T2

answer Blocked
(7500, 3000, 90)
('G1 R100 L100 T800\r\n', 'M3 H2 S0\r\n')
PreParse  M0 H2 T1

PostParse <Result ('1',) {}>
Message  M0 H2 T1

answer Timeout
Setting pos
G92 X7500 Y7700 A90 

{'h': 0, 'k': 0, 'e': 0, 'i': 1, 'f': 0, 'j': 0, 'b': 0, 'm': 0, 'c': 0, 'g': 0, 'l': 0, 'd': 0, 'a': 0}
Message  M0 X7487.11 Y7391.60 A92.00 T1 S256

Timeout
{'h': 0, 'k': 0, 'e': 0, 'i': 0, 'f': 0, 'j': 0, 'b': 0, 'm': 0, 'c': 0, 'g': 0, 'l': 0, 'd': 0, 'a': 0}
Message  M0 X7335.07 Y7423.38 A182.58 T1 S0

Timeout
('G1 R-320 L-300 T10000\r\n', 'M3 H2 S15\r\n')
PreParse  M0 H2 T2

PostParse <Result ('2',) {}>
Message  M0 H2 T2

answer Blocked
('G1 R-100 L-100 T1500\r\n', 'M3 H2 S0\r\n')
PreParse  M0 H2 T1

PostParse <Result ('1',) {}>
Message  M0 H2 T1

answer Timeout
Setting pos
G92 X7

KeyboardInterrupt: 

('G1 R640 L600 T20000\r\n', 'M3 H2 S15\r\n')
PreParse  M0 X4141.96 Y298.09 A-209.29 T1 S0

Message  M0 X4141.96 Y298.09 A-209.29 T1 S0

PreParse  M0 H2 T2

PostParse <Result ('2',) {}>
Message  M0 H2 T2

answer Blocked
('G1 R640 L600 T20000\r\n', 'M3 H2 S15\r\n')
PreParse  M0 H2 T2

PostParse <Result ('2',) {}>
Message  M0 H2 T2

answer Blocked
('G1 R100 L100 T800\r\n', 'M3 H2 S0\r\n')
PreParse  M0 H2 T1

PostParse <Result ('1',) {}>
Message  M0 H2 T1

answer Timeout
Setting pos
G92 X200 Y300 A180 

Message OK: D_Profil S=150 A=50 B=100 

{'h': 1, 'k': 0, 'e': 0, 'i': 0, 'f': 0, 'j': 1, 'b': 1, 'm': 0, 'c': 0, 'g': 1, 'l': 0, 'd': 0, 'a': 0}
Message  M0 X920.91 Y191.87 A171.62 T2 S706

Blocked
{'h': 0, 'k': 0, 'e': 0, 'i': 0, 'f': 0, 'j': 1, 'b': 0, 'm': 0, 'c': 0, 'g': 0, 'l': 0, 'd': 0, 'a': 0}
Message  M0 X1615.33 Y95.92 A156.49 T1 S512

Timeout
{'h': 0, 'k': 0, 'e': 0, 'i': 0, 'f': 0, 'j': 0, 'b': 0, 'm': 0, 'c': 0, 'g': 0, 'l': 0, 'd': 0, 'a': 0}
Message  M0 X1638.14 Y251.47 A43.3

KeyboardInterrupt: 

In [30]:
robot_Instance.Start_Collector()


In [11]:
robot_Instance.Set_Robot_Position(7600,4500,0,Transmission=True)
robot_Instance.Set_Robot_Speed(500, Transmission= True)

In [13]:
robot_Instance.Go_To(7000,4500,1, Avoidance=False)

robot_Instance.Go_To(8500,5000,0, Avoidance=False)

{'k': 0, 'c': 0, 'm': 0, 'h': 0, 'g': 0, 'l': 0, 'b': 0, 'i': 0, 'd': 0, 'a': 0, 'j': 0, 'e': 0, 'f': 0}
 M0 X7364.14 Y4889.43 A133.84 T1 S0

Timeout
{'k': 0, 'c': 0, 'm': 0, 'h': 0, 'g': 0, 'l': 0, 'b': 0, 'i': 0, 'd': 0, 'a': 0, 'j': 0, 'e': 0, 'f': 0}
 M0 X7355.67 Y4937.30 A89.11 T1 S0

Timeout


In [7]:
G1_String='G1 R' + str(round(400))+' L'+str(round(460))+' T'+str(round(10000))+'\r\n'
M3_String='M3 H2 S15' +'\r\n'
All_Commands = (G1_String,M3_String)
#print(All_Commands)
robot_Instance.Send_Messages(All_Commands)
M0_State, Output=robot_Instance.Wait_For_M2_Answer(4)
print(M0_State, Output)


M0 H2 T2

True Blocked


In [31]:
import math
X,Y,A=robot_Instance.Get_Robot_Position()
X_Bottle, Y_Bottle= Bottle_X_Extended, Bottle_Y_Extended
Error_X=X_Bottle-X;
Error_Y=Y_Bottle-Y;
Angle_Des=math.atan2(Error_Y,Error_X)
Angle_Des2=Angle_Des-math.radians(A)
X_New=X_Bottle+math.cos((Angle_Des2))*800
Y_New=Y_Bottle+math.sin((Angle_Des2))*800
print(X,Y)
print(X_New,Y_New)

3990.39 7512.51
3792.3552380456076 7086.044330961104


In [40]:
import math
X,Y,A=4000,8000,90
X_Bottle, Y_Bottle= 4000,7800
Error_X=X_Bottle-X;
Error_Y=Y_Bottle-Y;
Angle_Des=math.atan2(Error_Y,Error_X)

X_New=X_Bottle+math.cos((Angle_Des))*800
Y_New=Y_Bottle+math.sin((Angle_Des))*800
print(X,Y)
print(X_New,Y_New)

2000 0
434.3145750507621 1565.685424949238


In [25]:
Init=[7500,3000,90,15]
robot_Instance.Set_Robot_Position(Init[0],Init[1],Init[2],Transmission=True)
robot_Instance.Start_Collector()
robot_Instance.Set_Robot_Speed(500, Transmission= True)
robot_Instance.Clear_Read_Buffer()
robot_Instance.Go_To_Speed(600,640,20000, Detection=True)#Follow X coordinate
robot_Instance.Go_To_Speed(600,640,20000, Detection=True)
robot_Instance.Go_To_Speed(100,100,800,X_Cal=7600,Y_Cal=200,A_Cal=0, Detection=False, Calibration=True)#Recalibrate Top of the ramp
robot_Instance.Go_To(7000,200,1,Path_Checking=False,Avoidance=False)#Backward
robot_Instance.Go_To(7800,1000,0,Path_Checking=False,Avoidance=False)#Turn left

robot_Instance.Go_To_Speed(600,640,20000, Detection=True)#Follow Y coordintae
robot_Instance.Go_To_Speed(600,640,20000, Detection=True)
print(robot_Instance.Get_Robot_Position())

robot_Instance.Go_To_Speed(100,100,800,X_Cal=7500,Y_Cal=7700,A_Cal=90, Detection=False, Calibration=True)#Recalibrate Top of the ramp, Step1
print('Speed Control Over')
robot_Instance.Disable_All_Sensors()
robot_Instance.Go_To(7500,7300,1,Path_Checking=False,Avoidance=False)#Backward
robot_Instance.Go_To(7200,7300,0,Path_Checking=False,Avoidance=False)#Turn Left
robot_Instance.Go_To_Speed(-400,-400,10000, Detection=True)#Go against wall
robot_Instance.Go_To_Speed(-200,-200,1000,X_Cal=7900,A_Cal=180, Detection=False, Calibration=True)#Recalibrate Corner paltform
robot_Instance.Go_To(5700,7750,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(5400,7000,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(6500,6600,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7000,7100,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(5600,7100,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7100,7700,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7300,7700,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To_Speed(200,200,10000, Detection=True)#Downhill
robot_Instance.Go_To_Speed(100,100,800,X_Cal=7600,Y_Cal=7600,A_Cal=0, Detection=False, Calibration=True)
robot_Instance.Go_To(7300,7950,1,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7650,7300,0,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To_Speed(110,100,15000, Detection=True)#Downhill
robot_Instance.Go_To_Speed(640,600,20000, Detection=True)
robot_Instance.Go_To_Speed(100,100,800,X_Cal=7800,Y_Cal=300,A_Cal=-90, Detection=False, Calibration=True)
robot_Instance.Go_To(7800,700,1,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To(7000,700,1,Path_Checking=False,Avoidance=False)
robot_Instance.Go_To_Speed(640,600,20000, Detection=True)#Recalibrate Origin
robot_Instance.Go_To_Speed(640,600,20000, Detection=True)#Recalibrate Origin

robot_Instance.Go_To_Speed(100,100,800,X_Cal=200,Y_Cal=300,A_Cal=180, Detection=False, Calibration=True)

robot_Instance.Reverse_Collector()
robot_Instance.Go_To(1000,300,1,Path_Checking=False,Avoidance=False)
time.sleep(10)
robot_Instance.Stop_Collector()


G92 X7500 Y3000 A90 

('G1 R600 L640 T20000\r\n', 'M3 H2 S15\r\n')
PreParse OK: D_Profil S=500 A=50 B=100 

Message OK: D_Profil S=500 A=50 B=100 

PreParse  M0 H2 T2

PostParse <Result ('2',) {}>
Message  M0 H2 T2

answer Blocked
('G1 R600 L640 T20000\r\n', 'M3 H2 S15\r\n')
PreParse  M0 H2 T2

PostParse <Result ('2',) {}>
Message  M0 H2 T2

answer Blocked
('G1 R100 L100 T800\r\n', 'M3 H2 S0\r\n')
PreParse  M0 H2 T1

PostParse <Result ('1',) {}>
Message  M0 H2 T1

answer Timeout
Setting pos
G92 X7600 Y200 A0 

{'h': 0, 'k': 0, 'e': 1, 'i': 1, 'f': 1, 'j': 0, 'b': 0, 'm': 0, 'c': 0, 'g': 0, 'l': 0, 'd': 0, 'a': 0}
Message  M0 X7220.58 Y194.99 A-1.22 T1 S304

Timeout
{'h': 0, 'k': 0, 'e': 1, 'i': 1, 'f': 1, 'j': 0, 'b': 0, 'm': 0, 'c': 0, 'g': 0, 'l': 0, 'd': 0, 'a': 0}
Message  M0 X7702.79 Y745.55 A80.94 T1 S304

Timeout
('G1 R600 L640 T20000\r\n', 'M3 H2 S15\r\n')
PreParse  M0 H2 T2

PostParse <Result ('2',) {}>
Message  M0 H2 T2

answer Blocked
('G1 R600 L640 T20000\r\n', 'M3 H2 S15\r

In [ ]:
robot_Instance.Set_Robot_Position(0,250,90,True)
robot_Instance.Set_Robot_Speed(150)
robot_Instance.Set_Max_Try_Single_Goal(12)

Data=robot_Instance.Get_Data_TCP()
if 

In [26]:
robot_Instance.Reverse_Collector()


In [45]:
robot_Instance.Send_Go_To_Coordinate([2000,1500,500,10000,0])

('G0 X2000 Y1500 T10000 R0\r\n', 'M201 H0 S500\r\n', 'M3 H3\r\n')
